In [34]:
import polars as pl

pl_DataFrame = pl.internals.dataframe.frame.DataFrame

## 関数

In [35]:
def add_date_from_unixtime(df: pl_DataFrame, unixtime_column: str = "UNIXTIME", date_column: str = "Date") -> pl_DataFrame:
    return df.with_columns((pl.col(unixtime_column)*1000).alias(date_column).cast(pl.Datetime).dt.with_time_unit('ms'))


In [53]:
df = pl.read_csv("sample.csv")
df = add_date_from_unixtime(df)
df.head()

UNIXTIME,hash[ip.src.upper16],hash[ip.src.32],ip.dst.lower16,tcp.dport,Date
f64,str,str,f64,i64,datetime[ms]
1.6410e9,"""2fe1ec63c455bd...","""457d5c7b1a91d2...",100.1,23,2021-12-31 15:00:00.120
1.6410e9,"""2d9e8afbdd75fd...","""39825211c3134d...",100.101,2323,2021-12-31 15:00:01.210
1.6410e9,"""1f5f57cbe46c47...","""2ee37d765230ea...",101.1,80,2021-12-31 15:00:02.210
1.6410e9,"""cf31089c853c78...","""ea9eb9ad3e94e5...",101.101,8080,2021-12-31 15:00:03.330
1.6410e9,"""7f10d3eecd32bf...","""717aecfa766c46...",100.102,443,2021-12-31 15:00:04.360


In [38]:
pl.Date?

Init signature: pl.Date(*args: 'Any', **kwargs: 'Any') -> 'PolarsDataType'
Docstring:      Calendar date type.
File:           ~/.cache/pypoetry/virtualenvs/godaddy-3UyvmeC2-py3.9/lib/python3.9/site-packages/polars/datatypes.py
Type:           DataTypeClass
Subclasses:     

In [85]:
from datetime import datetime

In [86]:
extract_from = datetime(year = 2021, month = 12, day = 31, hour = 15, minute = 0, second = 10)
extract_to = datetime(year = 2021, month = 12, day = 31, hour = 15, minute = 0, second = 30)

In [92]:
filterd_df = df.filter((pl.col("Date") >= extract_from) & (pl.col("Date") <= extract_to))
val = filterd_df.get_column("Date").min()

In [106]:
filterd_df.with_columns((pl.col("Date") - val).alias("deltatime"))

UNIXTIME,hash[ip.src.upper16],hash[ip.src.32],ip.dst.lower16,tcp.dport,Date,deltatime
f64,str,str,f64,i64,datetime[ms],duration[ms]
1.6410e9,"""2fe1ec63c455bd...","""457d5c7b1a91d2...",100.1,23,2021-12-31 15:00:10.120,0ms
1.6410e9,"""2d9e8afbdd75fd...","""39825211c3134d...",100.101,2323,2021-12-31 15:00:11.210,1s 90ms
1.6410e9,"""1f5f57cbe46c47...","""2ee37d765230ea...",101.1,80,2021-12-31 15:00:12.210,2s 90ms
1.6410e9,"""cf31089c853c78...","""ea9eb9ad3e94e5...",101.101,8080,2021-12-31 15:00:13.330,3s 210ms
1.6410e9,"""7f10d3eecd32bf...","""717aecfa766c46...",100.102,443,2021-12-31 15:00:14.360,4s 240ms
1.6410e9,"""2fe1ec63c455bd...","""457d5c7b1a91d2...",100.1,23,2021-12-31 15:00:15.120,5s
1.6410e9,"""2d9e8afbdd75fd...","""39825211c3134d...",100.101,2323,2021-12-31 15:00:16.210,6s 90ms
1.6410e9,"""1f5f57cbe46c47...","""2ee37d765230ea...",101.1,80,2021-12-31 15:00:17.210,7s 90ms
1.6410e9,"""cf31089c853c78...","""ea9eb9ad3e94e5...",101.101,8080,2021-12-31 15:00:18.330,8s 210ms


In [108]:
import matplotlib.pyplot as plt

In [ ]:
plt.